In [1]:
import pandas as pd
import numpy as np
import os

import ipywidgets as widgets
from ipywidgets import interact
import plotly.graph_objs as go

In [2]:
econDF = pd.DataFrame(columns=['When','Alberta', 'Sector'])

for files in os.listdir("Economic"):
    df = pd.read_csv("Economic/" + files)
    df['When'] = pd.to_datetime(df['When'])
    tempDF = df[["When", "Alberta"]] # Get When and Alberta columns
    if files != 'Population.csv':
        tempDF = tempDF.join(df.iloc[:,2], lsuffix='_left', rsuffix='_right') # Get second column
    else:
        tempDF['Pop'] = 'Population'
        
    tempDF.columns = ["When", "Alberta", "SubSector"]
    tempDF['Sector'] = files.split('.')[0]
    econDF = econDF.append(tempDF) # append to final df

<ipython-input-2-781fdabfc879>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempDF['Pop'] = 'Population'
<ipython-input-2-781fdabfc879>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempDF['Sector'] = files.split('.')[0]


In [3]:
econDF['Month'] = econDF['When'].dt.month
econDF['Day'] = econDF['When'].dt.day
econDF['Year'] = econDF['When'].dt.year

In [4]:
display(econDF.head())
display(econDF.tail())

,When,Alberta,Sector,SubSector,Month,Day,Year
0,1985-01-01,81.15,CattlePrice,"Slaughter, calves",1,1,1985
1,1985-01-01,69.88,CattlePrice,"Slaughter, cattle",1,1,1985
2,1985-01-01,69.53,CattlePrice,Hogs,1,1,1985
3,1985-02-01,82.94,CattlePrice,"Slaughter, calves",2,1,1985
4,1985-02-01,71.24,CattlePrice,"Slaughter, cattle",2,1,1985


,When,Alberta,Sector,SubSector,Month,Day,Year
4935,2021-07-01,1506.23,WeeklyIncome,Public administration,7,1,2021
4936,2021-07-01,1328.57,WeeklyIncome,Real estate and rental and leasing,7,1,2021
4937,2021-07-01,922.39,WeeklyIncome,Trade,7,1,2021
4938,2021-07-01,1361.01,WeeklyIncome,Transportation and warehousing,7,1,2021
4939,2021-07-01,1954.83,WeeklyIncome,Utilities,7,1,2021


In [6]:
graphDF = econDF.groupby(['Year', 'Month', 'Sector']).mean().drop('Day', axis = 1)

# Now let's explore the data with the help of a drop-down interactor.

# Build a bar chart that we'll then update with the help of a call-back function
bar = go.Bar()
fig_bar = go.FigureWidget(data=bar)
#fig_bar.update_yaxes(range=[0, 250])

month_labels = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

# A list passed to interact() will yield a drop-down interactor
@interact(year=[*range(max(econDF['Year']),min(econDF['Year']),-1)],
          sector = list(econDF['Sector2'].unique()))

def update_bar(sector, year=2019):
    graphDF =  econDF[econDF['Sector2'] == sector]
    graphDF = graphDF.groupby(['Year', 'Month']).mean().drop('Day', axis = 1)

    data = graphDF.loc[year].mean(axis=1,skipna=True)
    fig_bar.update_traces(x=pd.Series(data.index.values).values, #.map(month_labels).values,
                          y=data.values)
    fig_bar.update_layout(title_text="{0} results for {1}".format(sector, year))

fig_bar

interactive(children=(Dropdown(description='sector', options=('Slaughter, calves (CattlePrice)', 'Slaughter, c…

FigureWidget({
    'data': [{'type': 'bar',
              'uid': 'a37ce9c9-e554-459f-a8f7-ba406a8a476e',
     …

In [7]:
#econDF['Sector2'] = econDF['SubSector'] + " (" + econDF['Sector'] + ")"

graphDF2 = econDF.groupby(['Year', 'Month', 'Sector']).mean().drop('Day', axis = 1)

# Now let's explore the data with the help of a drop-down interactor.

# Build a bar chart that we'll then update with the help of a call-back function
bar = go.Scatter()
fig_bar = go.FigureWidget(data=bar)
#fig_bar.update_yaxes(range=[0, 250])

month_labels = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

# A list passed to interact() will yield a drop-down interactor
@interact(sector = list(econDF['Sector'].unique()))

def update_bar(sector):
    
    graphDF2 = econDF[econDF['Sector'] == sector]
    graphDF2 = graphDF2.groupby(['Year', 'SubSector']).mean().drop(['Day', 'Month'], axis = 1)
    graphDF2.reset_index(inplace = True)
    data = graphDF2
    fig_bar.update_traces(x=[max(data['Year']),max(data['Year'])],
                       y=[max(data['Alberta']),max(data['Alberta'])])
    
    for subsector in data['SubSector'].unique():
        data1 = data[data['SubSector'] == subsector]
        fig_bar.add_trace(
            go.Scatter(x=data1['Year'],
                       y=data1['Alberta'],
                      name = subsector))
        
    fig_bar.update_layout(title_text="{0} results".format(sector))
    fig_bar.update_layout(showlegend=False)

fig_bar

interactive(children=(Dropdown(description='sector', options=('CattlePrice', 'NetMigration', 'Population', 'Pr…

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'aeaed5b8-6e32-452b-935b-babd4cb6e31d',
 …